In [4]:
import json
import re
import requests
from bs4 import BeautifulSoup
from wikidata.client import Client
import pandas as pd

In [87]:
def extract_peak_data(file, essencial):
    with open(file, "r") as f:
        data = json.load(f)

    for i, peak in enumerate(data):
        r = requests.get(peak["url"]).text
        soup = BeautifulSoup(r, "html.parser")

        peak["id"] = int(re.search(r"cim_id:\s*([^\n\r]*)", r).group(1).replace("'", ""))
        peak["altitud"] = int(soup.select(f'#post-{peak["id"]} > div.col-12.section-cim-content > div > div > div.col-lg-5.mb-5.order-lg-2.order-1 > div.bg-primari.white.p-4 > div > div:nth-child(4)')[0].text.split()[0])
        peak["comarca"] = soup.select(f'#post-{peak["id"]} > div.col-12.section-cim-content > div > div > div.col-lg-5.mb-5.order-lg-2.order-1 > div.bg-primari.white.p-4 > div > div:nth-child(2)')[0].text.replace("\t", "").replace("\n", "").replace("\r", "")
        peak["utm_x"] = int(soup.select(f'#post-{peak["id"]} > div.col-12.section-cim-content > div > div > div.col-lg-5.mb-5.order-lg-2.order-1 > div.bg-primari.white.p-4 > div > div:nth-child(10)')[0].text)
        peak["utm_y"] = int(soup.select(f'#post-{peak["id"]} > div.col-12.section-cim-content > div > div > div.col-lg-5.mb-5.order-lg-2.order-1 > div.bg-primari.white.p-4 > div > div:nth-child(12)')[0].text)
        peak["essencial"] = essencial

        print(f"Pic {i+1}/{len(data)}: {peak['nombre']}")
    return data

In [124]:
essencials = extract_peak_data("100cims_feec_essencials.json", essencial=True)

Pic 1/150: El Cogulló de Cabra
Pic 2/150: Puig de Tretzevents
Pic 3/150: Torre de Madeloc
Pic 4/150: Cambra d\'Ase
Pic 5/150: Canigó
Pic 6/150: Carlit
Pic 7/150: Puig Peric
Pic 8/150: Roc de Madres
Pic 9/150: Torre d\'Eina
Pic 10/150: Casamanya Nord
Pic 11/150: Comapedrosa
Pic 12/150: Pic de la Serrera
Pic 13/150: Pic del Vent
Pic 14/150: Pic Negre d\'Envalira
Pic 15/150: Sant Sadurní de Gallifa
Pic 16/150: Tagamanent
Pic 17/150: Tristaina
Pic 18/150: Puigmal
Pic 19/150: Castellsapera
Pic 20/150: El Cogul
Pic 21/150: La Mola de Sant Llorenç del Munt
Pic 22/150: Les Agudes
Pic 23/150: Maubèrme
Pic 24/150: Montardo
Pic 25/150: Montcorbison
Pic 26/150: Montlude
Pic 27/150: Punta de Montmeneu
Pic 28/150: Pilar d\'Almenara
Pic 29/150: Puig de la Creu
Pic 30/150: Puig de les Morreres
Pic 31/150: Roques de Benet (el Castell)
Pic 32/150: Sant Miquel de Solterra
Pic 33/150: Santa Bàrbara
Pic 34/150: Taga
Pic 35/150: Tossal de la Creu
Pic 36/150: Tuc de Molières
Pic 37/150: Tuc deth Port de Viel

In [88]:
no_essencials = extract_peak_data("100cims_feec_no_essencials.json", essencial=False)

Pic 1/372: la Soca
Pic 2/372: Piló de Bellmaig
Pic 3/372: Torre de la Maçana
Pic 4/372: Sant Martí de la Roca
Pic 5/372: la Serra (Vingrau)
Pic 6/372: Puig de la Collada Verda
Pic 7/372: Pic de Gallinassa
Pic 8/372: Santa Anna dels Quatre Termes
Pic 9/372: Puig de la Pelada
Pic 10/372: Pic de Coma Mitjana
Pic 11/372: Torre de Goà
Pic 12/372: Puig del Pam
Pic 13/372: Pic dels Pedrons
Pic 14/372: Puig Occidental de Coll Roig
Pic 15/372: Puig de Font Viva
Pic 16/372: Puig de la Cometa
Pic 17/372: el Punxó
Pic 18/372: Pic dels Pessons o Gargantillar
Pic 19/372: Pic de Cabanyó
Pic 20/372: Pic Alt del Cubil
Pic 21/372: Pic de la Coma de Varilles
Pic 22/372: Pic del Pla de l\'Estany
Pic 23/372: Alt de la Capa
Pic 24/372: el Sui
Pic 25/372: Turó de l\'Home
Pic 26/372: Turó de Montcada
Pic 27/372: Castell de Bocs
Pic 28/372: Tuc de Sarrahèra
Pic 29/372: Tuc des Armèros
Pic 30/372: Tuc d\'Arres
Pic 31/372: Montanha d\'Uishèra
Pic 32/372: Tuc de Crabèra
Pic 33/372: Malh de Bolard
Pic 34/372: Tuc 

In [107]:
cent_cims = essencials + no_essencials

with open("100cims.json", "w") as f:
    f.write(json.dumps(cent_cims, ensure_ascii=False, indent=4))

In [20]:
with open("100cims.json") as f:
    cent_cims = json.load(f)

In [3]:
with open("100cims_wikipedia.json") as f:
    cent_cims_w = json.load(f)

len(cent_cims_w["results"]["bindings"])

loc_w = list()
for item in cent_cims_w["results"]["bindings"]:
    lng, lat = item["location"]["value"].replace("Point(", "").replace(")", "").split()
    item["location"]["value"] = {"lat": float(lat), "lng": float(lng)}

In [4]:
cent_cims_w["results"]["bindings"][0]

{'q': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q599722'},
 'location': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral',
  'type': 'literal',
  'value': {'lat': 40.803130113, 'lng': 0.343121075}},
 'image': {'type': 'uri',
  'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Mont%20Caro%20%28enhanced%29.jpg'},
 'commonscat': {'type': 'literal', 'value': 'Mont Caro'},
 'qLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Mont Caro'},
 'desc': {'xml:lang': 'en', 'type': 'literal', 'value': 'mountain'}}

In [5]:
for cim_w in cent_cims_w["results"]["bindings"]:
    print(cim_w["q"]["value"])

http://www.wikidata.org/entity/Q599722
http://www.wikidata.org/entity/Q948850
http://www.wikidata.org/entity/Q954784
http://www.wikidata.org/entity/Q968445
http://www.wikidata.org/entity/Q1141050
http://www.wikidata.org/entity/Q1269228
http://www.wikidata.org/entity/Q1537733
http://www.wikidata.org/entity/Q1844624
http://www.wikidata.org/entity/Q2209719
http://www.wikidata.org/entity/Q2476121
http://www.wikidata.org/entity/Q2693141
http://www.wikidata.org/entity/Q2717368
http://www.wikidata.org/entity/Q2934685
http://www.wikidata.org/entity/Q2984642
http://www.wikidata.org/entity/Q3030915
http://www.wikidata.org/entity/Q3078719
http://www.wikidata.org/entity/Q3087278
http://www.wikidata.org/entity/Q3100806
http://www.wikidata.org/entity/Q3138291
http://www.wikidata.org/entity/Q3176503
http://www.wikidata.org/entity/Q3212922
http://www.wikidata.org/entity/Q3382362
http://www.wikidata.org/entity/Q3382364
http://www.wikidata.org/entity/Q3382368
http://www.wikidata.org/entity/Q3382387
http

In [6]:
count = 0
for i, cim in enumerate(cent_cims):
    lat, lng = cim["lat"], cim["lng"]
    for cim_w in cent_cims_w["results"]["bindings"]:
        lat_w, lng_w = cim_w["location"]["value"]["lat"], cim_w["location"]["value"]["lng"]
        if abs(lat - lat_w) <= 0.00001 and abs(lng_w - lng) <= 0.00001:
            count += 1
            print(f"Match for {cim['nom']}. {count}/{len(cent_cims)} (evaluated {i + 1}/{len(cent_cims)}).")
            cim["wikidata"] = cim_w["q"]["value"]
            try:
                cim["image"] = cim_w["image"]["value"]
            except KeyError:
                cim["image"] = None

Match for El Cogulló de Cabra. 1/522 (evaluated 1/522).
Match for Puig de Tretzevents. 2/522 (evaluated 2/522).
Match for Torre de Madeloc. 3/522 (evaluated 3/522).
Match for Cambra d\'Ase. 4/522 (evaluated 4/522).
Match for Canigó. 5/522 (evaluated 5/522).
Match for Puig Peric. 6/522 (evaluated 7/522).
Match for Roc de Madres. 7/522 (evaluated 8/522).
Match for Torre d\'Eina. 8/522 (evaluated 9/522).
Match for Casamanya Nord. 9/522 (evaluated 10/522).
Match for Comapedrosa. 10/522 (evaluated 11/522).
Match for Pic de la Serrera. 11/522 (evaluated 12/522).
Match for Pic del Vent. 12/522 (evaluated 13/522).
Match for Pic Negre d\'Envalira. 13/522 (evaluated 14/522).
Match for Sant Sadurní de Gallifa. 14/522 (evaluated 15/522).
Match for Tagamanent. 15/522 (evaluated 16/522).
Match for Tristaina. 16/522 (evaluated 17/522).
Match for Puigmal. 17/522 (evaluated 18/522).
Match for Castellsapera. 18/522 (evaluated 19/522).
Match for El Cogul. 19/522 (evaluated 20/522).
Match for La Mola de S

In [179]:
cent_cims

[{'nom': 'El Cogulló de Cabra',
  'lat': 41.4290355033,
  'lng': 1.30575794888,
  'url': 'https://www.feec.cat/activitats/100-cims/cim/el-cogullo-de-cabra/',
  'id': 639,
  'altitud': 881,
  'comarca': 'Alt Camp',
  'utm_x': 358437,
  'utm_y': 4587771,
  'essencial': True,
  'wikidata': 'http://www.wikidata.org/entity/Q17481698',
  'image': 'http://commons.wikimedia.org/wiki/Special:FilePath/El%20Cogull%C3%B3%20des%20de%20Montblanc%20P1250216.jpg'},
 {'nom': 'Puig de Tretzevents',
  'lat': 42.492101894,
  'lng': 2.4683643295,
  'url': 'https://www.feec.cat/activitats/100-cims/cim/puig-de-tretzevents/',
  'id': 637,
  'altitud': 2731,
  'comarca': 'Conflent, Vallespir',
  'utm_x': 456312,
  'utm_y': 4704553,
  'essencial': True,
  'wikidata': 'http://www.wikidata.org/entity/Q7838888',
  'image': 'http://commons.wikimedia.org/wiki/Special:FilePath/Cim%20de%20Tretzevents.jpg'},
 {'nom': 'Torre de Madeloc',
  'lat': 42.4904284262,
  'lng': 3.07511682009,
  'url': 'https://www.feec.cat/acti

In [14]:
for cim in cent_cims:
    if not "wikidata" in cim:
        print(cim["nom"])

In [19]:
for cim in cent_cims:
    if "imatge" in cim:
        cim.pop("imatge")

In [50]:
count = 0
for i, cim in enumerate(cent_cims):
    client = Client()
    entity = client.get(cim["wikidata"].split("/")[-1])
    if "P1296" in entity.attributes["claims"]:
        ec_id = entity.attributes["claims"]["P1296"][0]["mainsnak"]["datavalue"]["value"]
        r = requests.get(f"https://www.enciclopedia.cat/gran-enciclopedia-catalana/ec-gec-{ec_id}.xml").text
        soup = BeautifulSoup(r, "html.parser")
        cim["description"] = soup.select("#block-views-block-fitxa-tecnica-block-11 > div > div > div > div > div > div.views-field.views-field-field-brief > div > p > def")[0].text
        count += 1
        print(f"Match for {cim['nom']}. {count}/{len(cent_cims)} (evaluated {i + 1}/{len(cent_cims)}).")
    else:
        cim["description"] = None

IndexError: list index out of range

In [58]:
entity = client.get("Q7838888")
entity.attributes["claims"]["P1296"][0]["mainsnak"]["datavalue"]["value"]
entity.attributes["sitelinks"]["cawiki"]["url"]

'https://ca.wikipedia.org/wiki/Puig_dels_Tres_Vents'

In [54]:
r = requests.get("https://www.enciclopedia.cat/gran-enciclopedia-catalana/ec-gec-0298060").headers
soup = BeautifulSoup(r, "html.parser")
soup.select("#block-views-block-fitxa-tecnica-block-11 > div > div > div > div > div > div.views-field.views-field-field-brief > div > p > def")[0].text

IndexError: list index out of range

In [53]:
requests.get("https://www.enciclopedia.cat/gran-enciclopedia-catalana/ec-gec-0298060").text

'<!DOCTYPE html>\n<html lang="ca" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">\n  <head>\n    <meta charset="utf-8" />\n<script async src="https://www.googletagmanager.com/gtag/js?id=G-25SGBGPBS0" type="application/json" id="cookies_google_analytics_tracking_file"></script>\n<script type="application/json" id="cookies_google_analytics_tracking_script">window.dataLayer = window.dataLayer || [];function gtag(){dataLayer.push(arguments)};gtag("js", new Date());gtag("set", "developer_id.dMDhkMT", true);gtag("config", "G-25SGBGPBS0", {"groups":"default","page_location":"/404.html?page=" + document.location.pathname + document.location.search + "&from=" + doc

In [60]:
requests.get("https://www.enciclopedia.cat/gran-enciclopedia-catalana/ec-gec-0298060.xml").headers

{'Date': 'Thu, 24 Nov 2022 20:59:11 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '62292', 'Connection': 'keep-alive', 'cache-control': 'max-age=300, public', 'x-drupal-dynamic-cache': 'HIT', 'link': '<https://www.enciclopedia.cat/>; rel="canonical", <https://www.enciclopedia.cat/>; rel="shortlink", <https://www.enciclopedia.cat/>; rel="canonical", <https://www.enciclopedia.cat/>; rel="shortlink"', 'x-ua-compatible': 'IE=edge', 'content-language': 'ca', 'x-frame-options': 'SAMEORIGIN', 'expires': 'Sun, 19 Nov 1978 05:00:00 GMT', 'vary': 'Cookie', 'x-generator': 'Drupal 8 (https://www.drupal.org)', 'x-drupal-cache': 'MISS', 'last-modified': 'Thu, 24 Nov 2022 20:57:46 GMT', 'etag': '"1669323466"', 'Age': '85', 'X-Content-Type-Options': 'nosniff', 'X-Clara-WADP': '560266551 571360370', 'Server': 'wadp2', 'X-WADP-Cache': 'HIT'}

In [21]:
for cim in cent_cims:
    cim["comarca"] = cim["comarca"].split(", ")

In [22]:
cent_cims

[{'nom': 'El Cogulló de Cabra',
  'lat': 41.4290355033,
  'lng': 1.30575794888,
  'url': 'https://www.feec.cat/activitats/100-cims/cim/el-cogullo-de-cabra/',
  'id': 639,
  'altitud': 881,
  'comarca': ['Alt Camp'],
  'utm_x': 358437,
  'utm_y': 4587771,
  'essencial': True,
  'wikidata': 'http://www.wikidata.org/entity/Q17481698',
  'image': 'http://commons.wikimedia.org/wiki/Special:FilePath/El%20Cogull%C3%B3%20des%20de%20Montblanc%20P1250216.jpg'},
 {'nom': 'Puig de Tretzevents',
  'lat': 42.492101894,
  'lng': 2.4683643295,
  'url': 'https://www.feec.cat/activitats/100-cims/cim/puig-de-tretzevents/',
  'id': 637,
  'altitud': 2731,
  'comarca': ['Conflent', 'Vallespir'],
  'utm_x': 456312,
  'utm_y': 4704553,
  'essencial': True,
  'wikidata': 'http://www.wikidata.org/entity/Q7838888',
  'image': 'http://commons.wikimedia.org/wiki/Special:FilePath/Cim%20de%20Tretzevents.jpg'},
 {'nom': 'Torre de Madeloc',
  'lat': 42.4904284262,
  'lng': 3.07511682009,
  'url': 'https://www.feec.ca

In [23]:
with open("100cims.json", "w") as f:
    f.write(json.dumps(cent_cims, ensure_ascii=False, indent=4))

In [38]:
from itertools import chain

df = pd.read_json("100cims.json")
total_comarques = list(set(chain.from_iterable(df["comarca"].values.tolist())))

In [40]:
print(len(total_comarques))

48


In [7]:
with open("centcims.json") as f:
    cent_cims = json.load(f)

In [10]:
cent_cims[0]["url"]["wikidata"]

'http://www.wikidata.org/entity/Q17481698'

In [22]:
from wikidata.client import Client


client = Client()
entity = client.get("Q17481698", load=True)
image_prop = client.get("P18")
image = entity[image_prop]
image.image_url

'https://upload.wikimedia.org/wikipedia/commons/b/bd/El_Cogull%C3%B3_des_de_Montblanc_P1250216.jpg'

In [28]:
count = 0
for i, cim in enumerate(cent_cims):
    client = Client()
    entity = client.get(cim["url"]["wikidata"].split("/")[-1], load=True)
    try:
        image_prop = client.get("P18")
        image = entity[image_prop]
        cim["images"] = [image.image_url]
        count += 1
        print(f"Match for {cim['name']}. {count}/{len(cent_cims)} (evaluated {i + 1}/{len(cent_cims)}).")
    except KeyError:
        cim["images"] = []

cent_cims

Match for El Cogulló de Cabra. 1/522 (evaluated 1/522).
Match for Puig de Tretzevents. 2/522 (evaluated 2/522).
Match for Torre de Madeloc. 3/522 (evaluated 3/522).
Match for Cambra d\'Ase. 4/522 (evaluated 4/522).
Match for Canigó. 5/522 (evaluated 5/522).
Match for Carlit. 6/522 (evaluated 6/522).
Match for Puig Peric. 7/522 (evaluated 7/522).
Match for Roc de Madres. 8/522 (evaluated 8/522).
Match for Torre d\'Eina. 9/522 (evaluated 9/522).
Match for Casamanya Nord. 10/522 (evaluated 10/522).
Match for Comapedrosa. 11/522 (evaluated 11/522).
Match for Pic de la Serrera. 12/522 (evaluated 12/522).
Match for Pic del Vent. 13/522 (evaluated 13/522).
Match for Pic Negre d\'Envalira. 14/522 (evaluated 14/522).
Match for Sant Sadurní de Gallifa. 15/522 (evaluated 15/522).
Match for Tagamanent. 16/522 (evaluated 16/522).
Match for Tristaina. 17/522 (evaluated 17/522).
Match for Puigmal. 18/522 (evaluated 18/522).
Match for Castellsapera. 19/522 (evaluated 19/522).
Match for El Cogul. 20/52

[{'name': 'El Cogulló de Cabra',
  'id': 639,
  'coords': {'latlng': {'lat': 41.4290355033, 'lng': 1.3057579489},
   'utm': {'x': 358437, 'y': 4587771}},
  'topography': {'territory': [{'name': 'Catalunya', 'id': 'CAT'}],
   'vegueria': [{'name': 'Camp', 'id': 3}],
   'comarca': [{'name': 'Alt Camp', 'id': 1}]},
  'essential': True,
  'url': {'feec': 'https://www.feec.cat/activitats/100-cims/cim/el-cogullo-de-cabra/',
   'wikidata': 'http://www.wikidata.org/entity/Q17481698'},
  'images': ['https://upload.wikimedia.org/wikipedia/commons/b/bd/El_Cogull%C3%B3_des_de_Montblanc_P1250216.jpg'],
  'height': 881},
 {'name': 'Puig de Tretzevents',
  'id': 637,
  'coords': {'latlng': {'lat': 42.492101894, 'lng': 2.4683643295},
   'utm': {'x': 456312, 'y': 4704553}},
  'topography': {'territory': [{'name': 'Catalunya Nord', 'id': 'NORD'},
    {'name': 'Catalunya Nord', 'id': 'NORD'}],
   'vegueria': [{'name': 'Catalunya Nord', 'id': 4},
    {'name': 'Catalunya Nord', 'id': 4}],
   'comarca': [{'

In [29]:
with open("centcims.json", "w") as f:
    f.write(json.dumps(cent_cims, ensure_ascii=False, indent=4))

In [25]:
requests.get("https://www.wikidata.org/w/api.php?format=json&action=wbgetclaims&property=P18&entity=Q17481698").json()

{'claims': {'P18': [{'mainsnak': {'snaktype': 'value',
     'property': 'P18',
     'hash': '155ef1d4f70da4bcb7c9dadaf2558c19f8471766',
     'datavalue': {'value': 'El Cogulló des de Montblanc P1250216.jpg',
      'type': 'string'},
     'datatype': 'commonsMedia'},
    'type': 'statement',
    'id': 'Q17481698$ABAA037F-8AA4-47CB-8FF3-75E19505CA11',
    'rank': 'normal',
    'references': [{'hash': 'b3fd5e254143d8b52cd96cff43af9af5e3a82358',
      'snaks': {'P143': [{'snaktype': 'value',
         'property': 'P143',
         'hash': 'f13c99a16441677ec69fcea23f221d866f1fc98f',
         'datavalue': {'value': {'entity-type': 'item',
           'numeric-id': 199693,
           'id': 'Q199693'},
          'type': 'wikibase-entityid'},
         'datatype': 'wikibase-item'}]},
      'snaks-order': ['P143']}]}]}}